N-Queens using three different crossing over techniques
1. Uniform CO
2. Shuffle CO
3. Three Parent CO

In [ ]:
import random
import numpy as np

def random_chromosome(size):
    return [random.randint(1, size) for _ in range(size)]


def fitness(chromosome):
    n = len(chromosome)
    horizontal_collisions = sum(chromosome.count(queen) - 1 for queen in chromosome) / 2
    diagonal_collisions = 0

    left_diagonal = [0] * (2 * n - 1)
    right_diagonal = [0] * (2 * n - 1)

    for i in range(n):
        left_diagonal_index = int(i + chromosome[i] - 1)  # Ensure integer index
        right_diagonal_index = int(n - 1 + i - chromosome[i])  # Ensure integer index
        left_diagonal[left_diagonal_index] += 1
        right_diagonal[right_diagonal_index] += 1

    for i in range(2 * n - 1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i] - 1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i] - 1
        diagonal_collisions += counter / (n - abs(i - n + 1))

    return int(maxFitness - (horizontal_collisions + diagonal_collisions))


def probability(chromosome, fitness):
    return fitness(chromosome) / maxFitness

def random_pick(population, probabilities):
    populationWithProbabilty = zip(population, probabilities)
    total = sum(w for c, w in populationWithProbabilty)
    r = random.uniform(0, total)
    upto = 0
    for c, w in zip(population, probabilities):
        if upto + w >= r:
            return c
        upto += w
    assert False, "Shouldn't get here"

# Uniform CrossOver
def uniform_crossover(arr1, arr2, crossover_rate):
    arr1 = np.array(arr1)
    arr2 = np.array(arr2)
    mask = np.random.choice([True, False], size=len(arr1), p=[crossover_rate, 1 - crossover_rate])
    temp = arr1[mask].copy()
    arr1[mask] = arr2[mask]
    arr2[mask] = temp
    return arr1.tolist(), arr2.tolist()

# Shuffle Crossover
def shuffle_crossover(arr1, arr2, crossover_rate):
    arr1 = np.array(arr1)
    arr2 = np.array(arr2)
    idx = np.random.choice(len(arr1), size=len(arr1), replace=False)
    mask = np.random.choice([True, False], size=len(arr1), p=[crossover_rate, 1 - crossover_rate])
    arr1[mask] = arr2[idx][mask]
    arr2[mask] = arr1[idx][mask]
    return arr1.tolist(), arr2.tolist()

# Three Parent Crossover
def three_parent_crossover(arr1, arr2, arr3, crossover_rate):
    arr1 = np.array(arr1)
    arr2 = np.array(arr2)
    arr3 = np.array(arr3)
    mask = np.random.choice([0, 1, 2], size=len(arr1))
    child1 = np.choose(mask, [arr1, arr2, arr3])

    # Generate a second child by selecting genes not present in the first child
    mask_complement = np.logical_not(mask.astype(bool))
    child2 = np.choose(mask_complement, [arr1, arr2, arr3])

    return child1.tolist(), child2.tolist()


def mutate(x):  #randomly changing the value of a random index of a chromosome
    n = len(x)
    c = random.randint(0, n - 1)
    m = random.randint(1, n)
    x[c] = m
    return x

def genetic_queen(population, fitness, crossover_method):
    mutation_probability = 0.03
    new_population = []
    probabilities = [probability(n, fitness) for n in population]
    for i in range(len(population)):
        x = random_pick(population, probabilities) #best chromosome 1
        y = random_pick(population, probabilities) #best chromosome 2

        if crossover_method == "Uniform":
            child1, child2 = uniform_crossover(x, y, crossover_rate) #creating two new chromosomes from the best 2 chromosomes
        elif crossover_method == "Shuffle":
            child1, child2 = shuffle_crossover(x, y, crossover_rate)
        elif crossover_method == "Three Parent":
            z = random_pick(population, probabilities) #best chromosome 3
            child1, child2 = three_parent_crossover(x, y, z, crossover_rate)
        else:
            raise ValueError("Invalid crossover method specified.")

        if random.random() < mutation_probability:
            child1 = mutate(child1)
            child2 = mutate(child2)

        new_population.append(child1)
        new_population.append(child2)
        if fitness(child1) == maxFitness or fitness(child2) == maxFitness: break
    return new_population

def print_chromosome(chrom):
    print("Chromosome = {},  Fitness = {}"
        .format(str(chrom), fitness(chrom)))

In [ ]:
if __name__ == "__main__":
    nq = int(input("Enter Number of Queens: ")) #say N = 8
    maxFitness = (nq*(nq-1))/2  # 8*7/2 = 28

    crossover_methods = ["Uniform", "Shuffle", "Three Parent"]
    crossover_method = input("Enter crossover method (Uniform, Shuffle, Three Parent): ")
    if crossover_method not in crossover_methods:
        raise ValueError("Invalid crossover method specified.")

    crossover_rate = float(input("Enter crossover rate: "))

    population = [random_chromosome(nq) for _ in range(100)]

    generation = 1

    while not maxFitness in [fitness(chrom) for chrom in population]:
        print("=== Generation {} ===".format(generation))
        population = genetic_queen(population, fitness, crossover_method)
        print("")
        print("Maximum Fitness = {}".format(max([fitness(n) for n in population])))
        generation += 1
    chrom_out = []
    print("Solved in Generation {}!".format(generation-1))
    for chrom in population:
        if fitness(chrom) == maxFitness:
            print("")
            print("One of the solutions: ")
            chrom_out = chrom
            print_chromosome(chrom)

    board = []

    for x in range(nq):
        board.append(["x"] * nq)

    for i in range(nq):
        board[nq-chrom_out[i]][i]="Q"

    def print_board(board):
        for row in board:
            print (" ".join(row))

    print()
    print_board(board)

Enter Number of Queens: 8
Enter crossover method (Uniform, Shuffle, Three Parent): Three Parent
Enter crossover rate: 0.7
=== Generation 1 ===

Maximum Fitness = 26
=== Generation 2 ===

Maximum Fitness = 27
=== Generation 3 ===

Maximum Fitness = 26
=== Generation 4 ===

Maximum Fitness = 27
=== Generation 5 ===

Maximum Fitness = 27
=== Generation 6 ===

Maximum Fitness = 27
=== Generation 7 ===

Maximum Fitness = 27
=== Generation 8 ===

Maximum Fitness = 27
=== Generation 9 ===

Maximum Fitness = 28
Solved in Generation 9!

One of the solutions: 
Chromosome = [6, 3, 7, 2, 8, 5, 1, 4],  Fitness = 28

x x x x Q x x x
x x Q x x x x x
Q x x x x x x x
x x x x x Q x x
x x x x x x x Q
x Q x x x x x x
x x x Q x x x x
x x x x x x Q x


Enter Number of Queens: 8
Enter crossover method (Uniform, Shuffle, Three Parent): Three Parent
Enter crossover rate: 0.7
=== Generation 1 ===

Maximum Fitness = 26
=== Generation 2 ===

Maximum Fitness = 27
=== Generation 3 ===

Maximum Fitness = 26
=== Generation 4 ===

Maximum Fitness = 27
=== Generation 5 ===

Maximum Fitness = 27
=== Generation 6 ===

Maximum Fitness = 27
=== Generation 7 ===

Maximum Fitness = 27
=== Generation 8 ===

Maximum Fitness = 27
=== Generation 9 ===

Maximum Fitness = 28
Solved in Generation 9!

One of the solutions:
Chromosome = [6, 3, 7, 2, 8, 5, 1, 4],  Fitness = 28

x x x x Q x x x
x x Q x x x x x
Q x x x x x x x
x x x x x Q x x
x x x x x x x Q
x Q x x x x x x
x x x Q x x x x
x x x x x x Q x

SHUFFLE CO
Enter Number of Queens: 8
Enter crossover method (Uniform, Shuffle, Three Parent): Shuffle
Enter crossover rate: 0.7
=== Generation 1 ===

Maximum Fitness = 26
=== Generation 2 ===

Maximum Fitness = 27
=== Generation 3 ===

Maximum Fitness = 27
=== Generation 4 ===

Maximum Fitness = 27
=== Generation 5 ===

Maximum Fitness = 27
=== Generation 6 ===

Maximum Fitness = 27
=== Generation 7 ===

Maximum Fitness = 27
=== Generation 8 ===

Maximum Fitness = 27
=== Generation 9 ===

Maximum Fitness = 28
Solved in Generation 9!

One of the solutions:
Chromosome = [7, 1, 3, 8, 6, 4, 2, 5],  Fitness = 28

x x x Q x x x x
Q x x x x x x x
x x x x Q x x x
x x x x x x x Q
x x x x x Q x x
x x Q x x x x x
x x x x x x Q x
x Q x x x x x x

UNIFORM
Enter Number of Queens: 8
Enter crossover method (Uniform, Shuffle, Three Parent): Uniform
Enter crossover rate: 0.3
=== Generation 1 ===

Maximum Fitness = 26
=== Generation 2 ===

Maximum Fitness = 26
=== Generation 3 ===

Maximum Fitness = 27
=== Generation 4 ===

Maximum Fitness = 27
=== Generation 5 ===

Maximum Fitness = 27
=== Generation 6 ===

Maximum Fitness = 27
=== Generation 7 ===

Maximum Fitness = 27
=== Generation 8 ===

Maximum Fitness = 27
=== Generation 9 ===

Maximum Fitness = 28
Solved in Generation 9!

One of the solutions:
Chromosome = [5, 7, 2, 6, 3, 1, 4, 8],  Fitness = 28

x x x x x x x Q
x Q x x x x x x
x x x Q x x x x
Q x x x x x x x
x x x x x x Q x
x x x x Q x x x
x x Q x x x x x
x x x x x Q x x